In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# import tensorflow and tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds

# import random stuff that hopefully?? is going to be useful
import numpy as np
import random
import time
from matplotlib import pyplot as plt

In [0]:
# define hyperparameters
batch_size = 64
learning_rate = 0.0002
n_z_input = 100

# number of epochs and iterations per epoch
train_epoch = 40
iterations_per_epoch = 300

In [0]:
# takes in a series of graph_images ([#images, :, :, 0]) and plot them
# each image is displayed as a 50 by 50 pixel image
# where images are laid out in rows of 10 images and columns of 20
def display_graph(graph_images, title, shape=(10, 20), image_size=(50, 50)):
    fig = plt.figure(figsize=image_size) # define figure
    plt.title(title) # define title
    plt.axis('off') # remove axis
    for i in range(0, shape[0] * shape[1]):
        img = graph_images[i, :, :, :].astype(np.int32) # added this
        fig.add_subplot(shape[0], shape[1], i + 1)
        plt.axis('off')
        plt.imshow(img, cmap="gray")
    plt.show()

## Model

In [0]:
def resnet_block(input, features):
    out1 = tf.layers.conv2d(input, features, (3, 3), (1, 1), 'same')
    out2 = tf.layers.conv2d(out1, features, (3, 3), (1, 1), 'same')
    return out2 + input

In [0]:
# generator and discriminator 
def generator(features, name, reuse=False):
    with tf.variable_scope('generator' + name, reuse=reuse):
        # tf.layers.conv2d_transpose
        l1 = tf.layers.conv2d_transpose(features, 512, [4, 4], strides=(1, 1), activation=tf.nn.relu, padding="VALID")
        l2 = tf.layers.conv2d_transpose(l1, 256, [4, 4], strides=(4, 4), activation=tf.nn.relu, padding="SAME")
        l3 = tf.layers.conv2d_transpose(l2, 128, [4, 4], strides=(2, 2), activation=tf.nn.relu, padding="SAME")
        conv4 = tf.layers.conv2d_transpose(l3, 1, [4, 4], strides=(2, 2), padding="SAME")
        # # kernels | kernel dimension | stride | padding | activation
        #  512        [4, 4]            (1, 1),   "VALID"   relu
        #  256        [4, 4]            (4, 4),   "SAME"    relu
        #  128        [4, 4]            (2, 2),   "SAME"    relu
        #  1          [4, 4]            (2, 2),   "SAME"    relu
        # conv3 is the output of the last conv2d_transpose layer
#         l1 = tf.layers.conv2d_transpose(features, 1024, 4, strides=1, activation="relu", padding="VALID")
#         # 512, (4, 4), stride = (2, 2), relu, padding="SAME"
#         l2 = tf.layers.conv2d_transpose(l1, 512, 4, strides=2, activation="relu", padding="SAME")
#         # 256, (4, 4), stride = (2, 2), relu, padding="SAME"
#         l3 = tf.layers.conv2d_transpose(l2, 256, 4, strides=2, activation="relu", padding="SAME")
#         # 128, (4, 4), stride = (2, 2), relu, padding="SAME"
#         l4 = tf.layers.conv2d_transpose(l3, 128, 4, strides=2, activation="relu", padding="SAME")
#         # 1 kernel, (4, 4), stride = 2, tanh, padding="SAME"
#         final = tf.layers.conv2d_transpose(l4, 512, 4, strides=2, activation="tanh", padding="SAME")
        return tf.nn.tanh(conv4)

In [0]:
def discriminator(features, name, reuse=False):
    with tf.variable_scope('discriminator' + name, reuse=reuse):
        # # kernels | kernel dimension | stride | padding | activation
        #  128        [4, 4]             (2, 2),  "SAME"    leaky_relu
        #  256        [4, 4]             (2, 2),  "SAME"    leaky_relu
        #  512        [4, 4]             (4, 4),  "SAME"    leaky_relu
        #  1024       [3, 3]             (1, 1),  "VALID"   leaky_relu
        l1 = tf.layers.conv2d(features, 128, [4, 4], strides=(2, 2), activation=tf.nn.leaky_relu, padding="SAME")
        l2 = tf.layers.conv2d(l1, 256, [4, 4], strides=(2, 2), activation=tf.nn.leaky_relu, padding="SAME")
        l3 = tf.layers.conv2d(l2, 512, [4, 4], strides=(4, 4), activation=tf.nn.leaky_relu, padding="SAME")
        conv4 = tf.layers.conv2d(l3, 1024, [3, 3], strides=(1, 1), activation=tf.nn.leaky_relu, padding="VALID")
        # conv4 is the output of the last conv2d_transpose layer
#         # 128 kernels, (4, 4) kernel size, stride 2, relu, padding = "SAME"
#         layer_1 = tf.layers.conv2d(features, 128, 4, strides=2, activation="relu", padding="SAME") 
#         # 256 kernels, (4, 4) kernel size, stride 2, relu, padding = "SAME"
#         layer_2 = tf.layers.conv2d(layer_1, 256, 4, strides=2, activation="relu", padding="SAME") 
#         # 512 kernels, (4, 4) kernel size, stride 2, relu, padding = "SAME"
#         layer_3 = tf.layers.conv2d(layer_2, 512, 4, strides=2, activation="relu", padding="SAME") 
#         # 1024 kernels, (4, 4) kernel size, stride 2, relu, padding = "SAME"
#         layer_4 = tf.layers.conv2d(layer_3, 1024, 4, strides=2, activation="relu", padding="SAME") 
#         # 1 kernel, (4, 4) kernel size, stride 1, relu, padding = "VALID"
#         final = tf.layers.conv2d(layer_4, 1, 4, strides=1, activation="relu", padding="VALID")
        flatten = tf.contrib.layers.flatten(conv4)
        logits = tf.layers.dense(flatten, 1)
        # use sigmoid to squash output into a probability
        output = tf.nn.sigmoid(logits) 
        
        return output, logits

## Placeholders

In [0]:
x1 = tf.placeholder(tf.float32, shape=(None, 256, 256, 3))
x2 = tf.placeholder(tf.float32, shape=(None, 256, 256, 3))
# z = tf.placeholder(tf.float32, shape=(None, 256, 256, 3)) # n_z_input))
# 3 was originally n_z_input and the 64s were 1s for z vector

In [7]:
# generator is generating an image (g is the fake image)
g1 = generator(x1, 'apple', False)
g2 = generator(x2, 'orange', False)
# discriminator is classifying real images 
# the first output is the probability and the second is the logits
# to feed into sigmoid_cross_entropy_with_logits
disc_real_1, disc_real_logits_1 = discriminator(x1, 'apple', False)
disc_fake_1, disc_fake_logits_1 = discriminator(g1, 'apple', True)

disc_real_2, disc_real_logits_2 = discriminator(x2, 'orange', False)
disc_fake_2, disc_fake_logits_2 = discriminator(g2, 'orange', True)

# get accuracy of the discriminator in
# predicting that the image is real or fake

# the goal is to get a list of 1 and 0 whether it predicted right or not
# and use tf.reduce_mean to get mean of 1 and 0 which returns a probability
# you can use > or < 0.5 to get true and false 
# whether it predicted right or not and use tf.cast to cast that boolean into tf.float32
real_accuracy_1 = tf.reduce_mean(tf.cast(disc_real_1 > 0.5, tf.float32)) # TODO
fake_accuracy_1 = tf.reduce_mean(tf.cast(disc_fake_1 < 0.5, tf.float32)) # TODO

real_accuracy_2 = tf.reduce_mean(tf.cast(disc_real_2 > 0.5, tf.float32))
fake_accuracy_2 = tf.reduce_mean(tf.cast(disc_fake_2 < 0.5, tf.float32))

Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


ValueError: ignored

Important! The labels are tf.ones([batch_size, 1]) instead of tf.ones([batch_size, 1, 1, 1]) because the output of the discriminator is a single probability i.e. [batch_size, 0.5] instead of [batch_size, 0.5, 1, 1] because its no longer the output of a conv layer but the output of a dense layer. 

In [0]:
# loss function for discriminator_1
disc_loss_real_1 = tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_real_logits_1, labels=tf.ones([batch_size, 1]))
disc_loss_real_1 = tf.reduce_mean(disc_loss_real_1)

disc_loss_fake_1 = tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_logits_1, labels=tf.zeros([batch_size, 1]))
disc_loss_fake_1 = tf.reduce_mean(disc_loss_fake_1)

disc_loss_total_1 = disc_loss_real_1 + disc_loss_fake_1

In [0]:
# loss function for discriminator_2
disc_loss_real_2 = tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_real_logits_2, labels=tf.ones([batch_size, 1]))
disc_loss_real_2 = tf.reduce_mean(disc_loss_real_2)

disc_loss_fake_2 = tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_logits_2, labels=tf.zeros([batch_size, 1]))
disc_loss_fake_2 = tf.reduce_mean(disc_loss_fake_2)
 
disc_loss_total_2 = disc_loss_real_2 + disc_loss_fake_2

In [0]:
# loss function for generator
gen_loss_1 = tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_logits_1, labels=tf.ones([batch_size, 1]))
gen_loss_1 = tf.reduce_mean(gen_loss_1)

In [0]:
# loss function for generator_2
gen_loss_2 = tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_fake_logits_2, labels=tf.ones([batch_size, 1]))
gen_loss_2 = tf.reduce_mean(gen_loss_2)

In [0]:
t_vars = tf.trainable_variables()
disc_var_1 = [var for var in t_vars if var.name.startswith('discriminatorapple')]
gen_var_1 = [var for var in t_vars if var.name.startswith('generatorapple')]

disc_var_2 = [var for var in t_vars if var.name.startswith('discriminatororange')] # HAVE TO CHANGE SOMETHING STILL

gen_var_2 = [var for var in t_vars if var.name.startswith('generatororange')]

In [0]:
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    gen_optimizer_1 = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(gen_loss_1, var_list=gen_var_1)
    disc_optimizer_1 = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(disc_loss_total_1, var_list=disc_var_1)
    gen_optimizer_2 = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(gen_loss_2, var_list=gen_var_2)
    disc_optimizer_2 = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(disc_loss_total_2, var_list=disc_var_2)
    real_fake_loss = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.5).minimize(## (real_apple - fake_apple)^2)

Load Data

In [0]:
dataset, metadata = tfds.load('cycle_gan/apple2orange', with_info=True, as_supervised=True)

In [0]:
apple_train_dataset, orange_train_dataset, apple_test_dataset, orange_test_dataset = datasets["TRAINA"], datasets["TRAINB"], datasets["TESTA"], datasets["TESTB"]

In [0]:
def apply_map(inputs):
    img = inputs['image']
    # 1. cast the img into float32
    img = tf.cast(img, tf.float32)
    img = tf.subtract(img, 0.5)
    img = tf.divide(img, 0.5)
    # 2. image values are from 0 to 1. convert to range -1 to 1
    # by subtracting by a decimal so its range -0.5 to 0.5 and
    # then divide by a decimal so its not -1 to 1
    # img = tf.math.tanh(img)
    # 3. use tf.image.resize to conver the image into a 64 by 64 image
    img = tf.image.resize(img, [256, 256])
    return img #tanh???

In [0]:
apple_train_dataset = apple_train_dataset.map(apply_map)
apple_train_dataset = apple_train_dataset.shuffle(1024)
apple_train_dataset = apple_train_dataset.batch(batch_size)

apple_iterator = apple_train_dataset.make_initializable_iterator()
apple_batch = apple_iterator.get_next()

orange_train_dataset = orange_train_dataset.map(apply_map)
orange_train_dataset = orange_train_dataset.shuffle(1024)
orange_train_dataset = orange_train_dataset.batch(batch_size)

orange_iterator = orange_train_dataset.make_initializable_iterator()
orange_batch = orange_iterator.get_next()

In [0]:
# z_test input into the generator with batch_size 10 
z_test = np.random.normal(0, 1, (10, 1, 1, n_z_input))

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize variables
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(train_epoch):
        print("Starting epoch: " + str(epoch))
        # initialize iterator
        sess.run(iterator.initializer)
        step = 0

        while True:
            try:
                tra_images = sess.run(batch) # grab the batch

                # makes sure the batch_size is 64
                if tra_images.shape[0] != 64:
                    break

                # do the generator separately?
                
                # then do the discriminator?
                    
                # z vector of size batch_size
                z_batch = np.random.normal(0, 1, (batch_size, 1, 1, n_z_input))
                
                acc_fake, acc_real, loss_d, _, loss_g, _ = sess.run(
                  [fake_accuracy, real_accuracy, disc_loss_total, disc_optimizer, gen_loss, gen_optimizer],
                  feed_dict={x: tra_images, z: z_batch})
                step += 1

                if step % 200 == 0:
                    generated_images = sess.run(g, feed_dict={z :z_test}) # CHECK THIS
                    display_graph(generated_images, "MNIST Images", (5, 2), image_size=(20, 20))
                print('Epoch: %d, Iteration: %d, loss_d: %.3f, loss_g: %.3f, acc_fake: %.3f, acc_real: %.3f' % (
                        epoch, step, loss_d, loss_g, acc_fake, acc_real))
                
            except tf.errors.OutOfRangeError:
                break